# This should be the final version of the Rossman script
The model is a ensemble of xgboosts with the same data.<br>
The data used includes:
    1. Simple aggregates for each store and initial data. Mostly without any preprocessing
    2. Weather, dates of football games
    3. Date derived features, so that the object will have features on how long is is till next promotion, till next holiday, etc.
    
Data cleaning includes deletion of days near holidays, as they add a lot of noise and are not relevant to the test set.

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
store = pd.read_csv('store.csv')
tr = pd.read_pickle('train_holidays_corrected')
te = pd.read_pickle('test_holidays_corrected')

## Preprocessing function
Left comments in Russian

In [5]:
def preprocess_train_data(train, test, store, scaler = None):
    """
    Препроцессинг данных
    Inputs: 
        train, test - пара датафреймов, уже разбитые на train и test. Так сделано, потому что на трейне должны 
        считаться агрегаты и не должно быть утечки информации
    
    Outputs: 
        train, test - перезаписанные датафреймы
    """
    from sklearn.preprocessing import LabelEncoder
    # Препроцессинг даты
    def preprocess_date(x):
        x['Date'] = pd.to_datetime(x['Date'], format = '%Y-%m-%d')
        x['day'] = x['Date'].dt.day
        x['year'] = x['Date'].dt.year
        x['month'] = x['Date'].dt.month
        x['dayofyear'] = x['Date'].dt.dayofyear
        x['weekofyear'] = x['Date'].dt.weekofyear
        return x
    
    train = preprocess_date(train)
    test = preprocess_date(test)
    
    train = train[(train.till_holiday>1) & (train.from_holiday>1)]
    
    # Расчет средних характеристик (возможно только на трейне, 
    # в других случаях буду цеплять к тесту через merge по идентификатору магазина)
    def preprocess_fixed_effects(x, l):
        for i in l:
            x = pd.merge(x, i, on = 'Store')
        return x

    def kurt(x):
        return pd.Series({'ku':x.Sales.kurtosis(axis = 0)})
    
    def sk(x):
        return pd.Series({'sk':x.Sales.skew()})
    
    def frac_mean(x):
        return np.mean(x['Sales'] / x['Customers'])
    
    def frac_median(x):
        return np.median(x['Sales'] / x['Customers'])
    
    def replace_promo(x):
        x.loc[:, 'PromoInterval_Feb,May,Aug,Nov'] = x.loc[:, ['PromoInterval_Feb,May,Aug,Nov', 'month']].prod(axis = 1) 
        x.loc[:, 'PromoInterval_Jan,Apr,Jul,Oct'] = x.loc[:, ['PromoInterval_Jan,Apr,Jul,Oct', 'month']].prod(axis = 1) 
        x.loc[:, 'PromoInterval_Mar,Jun,Sept,Dec'] = x.loc[:, ['PromoInterval_Mar,Jun,Sept,Dec', 'month']].prod(axis = 1) 
        x['PromoInterval_Feb,May,Aug,Nov'].replace({1: 2,
                                                     2: 1,
                                                     3: 2,
                                                     4: 2,
                                                     5: 1,
                                                     6: 2,
                                                     7: 2,
                                                     8: 1,
                                                     9: 2,
                                                     10: 2,
                                                     11: 1,
                                                     12: 2}, inplace = True)
        x['PromoInterval_Jan,Apr,Jul,Oct'].replace({1: 1,
                                                     2: 2,
                                                     3: 2,
                                                     4: 1,
                                                     5: 2,
                                                     6: 2,
                                                     7: 1,
                                                     8: 2,
                                                     9: 2,
                                                     10: 1,
                                                     11: 2,
                                                     12: 2}, inplace = True)
        x['PromoInterval_Mar,Jun,Sept,Dec'].replace({1: 2,
                                                     2: 2,
                                                     3: 1,
                                                     4: 2,
                                                     5: 2,
                                                     6: 1,
                                                     7: 2,
                                                     8: 2,
                                                     9: 1,
                                                     10: 2,
                                                     11: 2,
                                                     12: 1}, inplace = True)
        x.loc[(x.year<x.Promo2SinceYear) | ((x.year==x.Promo2SinceYear) & (x.weekofyear<x.Promo2SinceWeek)),
             ['PromoInterval_Mar,Jun,Sept,Dec', 'PromoInterval_Jan,Apr,Jul,Oct', 'PromoInterval_Feb,May,Aug,Nov']] = 0
        x.loc[:, 'Promo2_status'] = x[['PromoInterval_Mar,Jun,Sept,Dec', 'PromoInterval_Jan,Apr,Jul,Oct', 'PromoInterval_Feb,May,Aug,Nov']].max(axis = 1)
        return x
    merge_list = list()
    merge_list.append(train[(train.Open == 1) & (train.Sales > 0)].groupby('Store').Sales.mean().to_frame().reset_index().rename(columns = {'Sales': 'mean_Sales'}))
#     merge_list.append(train[(train.Open == 1) & (train.Sales > 0)].groupby('Store').Sales.median().to_frame().reset_index().rename(columns = {'Sales': 'median_Sales'}))
    merge_list.append(train[(train.Open == 1) & (train.Customers > 0)].groupby('Store').Customers.mean().to_frame().reset_index().rename(columns = {'Customers': 'mean_Customers'}))
#     merge_list.append(train[(train.Open == 1) & (train.Customers > 0)].groupby('Store').Customers.median().to_frame().reset_index().rename(columns = {'Customers': 'median_Customers'}))
    merge_list.append(train[(train.Open == 1) & (train.Customers > 0)].groupby('Store').apply(frac_mean).to_frame().reset_index().rename(columns = {0: 'mean_frac'}))
    merge_list.append(train[(train.Open == 1) & (train.Customers > 0)].groupby('Store').apply(kurt).reset_index())
    merge_list.append(train[(train.Open == 1) & (train.Customers > 0)].groupby('Store').apply(sk).reset_index())
#     merge_list.append(train[(train.Open == 1) & (train.Customers > 0)].groupby('Store').apply(frac_median).to_frame().reset_index().rename(columns = {0: 'median_frac'}))
    def create_store_feat(data):
        data.loc[:,'day_from_beg'] = (data.year - 2013)*365 + data.dayofyear
        store_feat = data.groupby('Store').\
            apply(lambda x: pd.Series([np.corrcoef(x.day_from_beg, x.Sales)[0,1]*x.Sales.std()/x.day_from_beg.std(),
                                                         x.Sales.std()], index=['trend', 'sales_std']))
        return store_feat.reset_index()
    merge_list.append(create_store_feat(train[(train.Open == 1) & (train.Sales > 100)]))
    
    train = preprocess_fixed_effects(train, merge_list)
    test = preprocess_fixed_effects(test, merge_list)
    
    month_mean_sales = train.groupby(['Store', 'month'])['Sales'].mean().reset_index().rename(columns = {'Sales': 'mean_month_sales'})
    train = pd.merge(train, month_mean_sales, on = ['Store', 'month'])
    test = pd.merge(test, month_mean_sales, on = ['Store', 'month'])
    # Добавляю таблицу Store 
    store['CompetitionDistance'].fillna(75860., inplace = True)
    train = train.merge(store[['Store', 'StoreType', 'Assortment', 'CompetitionDistance', 'Promo2', 'PromoInterval', 'CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth', 'Promo2SinceYear', 'Promo2SinceWeek']], on = 'Store', how = 'left')
    test = test.merge(store[['Store', 'StoreType', 'Assortment', 'CompetitionDistance', 'Promo2', 'PromoInterval', 'CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth', 'Promo2SinceYear', 'Promo2SinceWeek']], on = 'Store', how = 'left')
    # делаем дамми. 
    train.StateHoliday.replace(0, '0', inplace = True)
    test.StateHoliday.replace(0, '0', inplace = True)
    le = LabelEncoder()
    for i in ['StateHoliday', 'StoreType', 'Assortment']:
        le.fit(train[i].append(test[i]))
        train.loc[:, i] = le.transform(train[i])
        test.loc[:, i] = le.transform(test[i])
    train = pd.get_dummies(train, columns = ['PromoInterval'])
    test = pd.get_dummies(test, columns = ['PromoInterval'])


    train['competitor_time_distance'] = ((train['year'] - train['CompetitionOpenSinceYear'])*12 + (train['month'] - train['CompetitionOpenSinceMonth']))
    train['competitor_exists'] = (train['competitor_time_distance']>0).astype(int)
    train['competitor_time_distance'] = train['competitor_time_distance'].apply(lambda x: max(0, x))
    
    test['competitor_time_distance'] = ((test['year'] - test['CompetitionOpenSinceYear'])*12 + (test['month'] - test['CompetitionOpenSinceMonth']))
    test['competitor_exists'] = (test['competitor_time_distance']>0).astype(int)
    test['competitor_time_distance'] = test['competitor_time_distance'].apply(lambda x: max(0, x))
    
    # Часть выверки одинаковости колонок в тесте и трейне
    train_cols = list(train.columns)
    test_cols = list(test.columns)
    if 'Sales' in train_cols:
        train_cols.remove('Sales')
    if 'Sales' in test_cols:
        test_cols.remove('Sales')
    if 'Customers' in train_cols:
        train_cols.remove('Customers')
#         del train['Customers']
    test_cols = list(test_cols)
    if 'Customers' in test_cols:
        test_cols.remove('Customers')
#         del test['Customers']
    if 'Id' in test_cols:
        test_cols.remove('Id')
    # На всякий случай выверяем, что колонки в тесте и трейне одинаковы
    if sorted(train_cols) != sorted(test_cols):
        for i in set(train_cols).difference(set(test_cols)):
            test.loc[:, i] = 0
        for i in set(test_cols).difference(set(train_cols)):
            train.loc[:, i] = 0
    train = replace_promo(train)
    test = replace_promo(test)
#     train.drop(labels = ['Store', 'Date'], axis = 1, inplace = True)
#     test.drop(labels = ['Store', 'Date'], axis = 1, inplace = True)
    train.drop(labels = ['Date'], axis = 1, inplace = True)
    test.drop(labels = ['Date'], axis = 1, inplace = True)
    
    all_cols = list(set(train_cols).union(set(test_cols)))
    all_cols.append('Promo2_status')
#     all_cols.remove('Store')
    all_cols.remove('Date')
    all_cols.remove('Open')
    all_cols.remove('CompetitionOpenSinceYear')
    all_cols.remove('CompetitionOpenSinceMonth')
    test.fillna(1, inplace = True)
    train.loc[:, 'till_holiday'] = train.loc[:, 'till_holiday'].apply(lambda x: min(x, 14))
    train.loc[:, 'from_holiday'] = train.loc[:, 'from_holiday'].apply(lambda x: min(x, 14))
    test.loc[:, 'till_holiday'] = test.loc[:, 'till_holiday'].apply(lambda x: min(x, 14))
    test.loc[:, 'from_holiday'] = test.loc[:, 'from_holiday'].apply(lambda x: min(x, 14))
    if scaler:
        train.fillna(0, inplace = True)
        test.fillna(1, inplace = True)
        scaler.fit(train.loc[train.Open == 1, all_cols].append(test.loc[test.Open == 1, all_cols], ignore_index = True))
        train.loc[:, all_cols] = scaler.transform(train[all_cols])
        test.loc[:, all_cols] = scaler.transform(test[all_cols])
        
    train = train[train.Open == 1]
    train = train[train.Sales > 0]
    
    return train, test, all_cols

def make_train_test(x, size = 45, same_period = False):
    if same_period:
        return x[(x.Date < pd.to_datetime('2013-08-01')) | (x.Date > pd.to_datetime('2013-09-17'))], x[(x.Date >=pd.to_datetime('2013-08-01')) & (x.Date <= pd.to_datetime('2013-09-17'))]
    else:
        max_date = x.Date.max()
        return x[x.Date < (max_date - np.timedelta64(size, 'D'))], x[x.Date >= (max_date - np.timedelta64(size, 'D'))] 

def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe

def rmspe_xg(yhat, y):
    # y = y.values
    y = y.get_label()
    y = np.exp(y) - 1
    yhat = np.exp(yhat) - 1
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y - yhat)**2))
    return "rmspe", rmspe

In [6]:
from sklearn.preprocessing import StandardScaler
validation = True
test_stores = te.Store.unique()
if validation:
    tr.Date = pd.to_datetime(tr.Date, format = '%Y-%m-%d')
    tr, te = make_train_test(tr, 47, same_period = False)
scaler = StandardScaler()
X_train, X_test, features = preprocess_train_data(tr, te, store)
# features.remove('Store')

/opt/conda/Anaconda3/lib/python3.4/site-packages/pandas/core/indexing.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/Anaconda3/lib/python3.4/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


## Add external data

In [7]:
states = pd.read_csv('external/store_states.csv')
weather = pd.read_pickle('external/weather_ready')

use_weather_cols = [name for name in weather.columns if not ('Max_' in name or 'Min_' in name)]
new_weather_cols = use_weather_cols.copy()
for col in use_weather_cols:
    if any(weather[col].isnull()):
        weather[col+'_nan'] = weather[col].isnull().astype(np.int8)
        new_weather_cols.append(col+'_nan')

weather = weather[new_weather_cols]
weather.fillna(weather.mean(), inplace=True)

weather['Date'] = pd.to_datetime(weather.Date, format='%Y-%m-%d')

weather['day'] = weather.Date.dt.day
weather['month'] = weather.Date.dt.month
weather['year'] = weather.Date.dt.year

relevant = ['State', 'Events', 'Mean_TemperatureC', 'Mean_Sea_Level_PressurehPa', 'Precipitationmm', 'year', 'month', 'day']
weather = weather[relevant]



X_train = X_train.merge(states, on='Store', how='inner')
X_test = X_test.merge(states, on='Store', how='inner')

X_train = X_train.merge(weather, on=['State', 'year', 'month', 'day'])
X_test = X_test.merge(weather, on=['State', 'year', 'month', 'day'])

X_train.Events.fillna('nan', inplace=True)
X_test.Events.fillna('nan', inplace=True)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(np.hstack([X_train.State.values, X_test.State.values]))
X_train.State = le.transform(X_train.State.values)
X_test.State = le.transform(X_test.State.values)

le = LabelEncoder()
le.fit(np.hstack([X_train.Events.values, X_test.Events.values]))
X_train.Events = le.transform(X_train.Events.values)
X_test.Events = le.transform(X_test.Events.values)


championship = pd.DataFrame([[1,16,6,2014],
                            [1,21,6,2014],
                            [1,26,6,2014],
                            [1,30,6,2014],
                            [1,4,7,2014],
                            [1,8,7,2014],
                            [1,13,7,2014]], columns = ['is_match', 'day', 'month', 'year'])

def add_champ(X_train):
    X_train = X_train.merge(championship, on =  ['day', 'month', 'year'], how = 'left').fillna(0)

    X_train['is_champ'] = 0
    X_train['is_champ'][(X_train.year == 2014) & 
                        (((X_train.month == 6) & (X_train.day >= 16)) | ((X_train.month == 7) & (X_train.day <= 13)))] = 1
    return X_train

X_train = add_champ(X_train)
X_test = add_champ(X_test)

/opt/conda/Anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
X_train = X_train[(X_train.till_holiday>1) & (X_train.from_holiday>1)]

## Deletion of some features. It is easier to make here instead of changing preprocessing function (it works fast)

In [206]:
features.append('skew')
features.append('kurt')
features.remove('sk')
features.remove('ku')

In [10]:
l =( ['till_closed', 
     'from_promo', 'from_holiday',
     'from_school_holiday', 'till_promo', 'till_school', 'till_holiday', 
    'from_closed'] 
   +  ['PromoInterval_Feb,May,Aug,Nov', 'PromoInterval_Mar,Jun,Sept,Dec', 'PromoInterval_Jan,Apr,Jul,Oct']  
        +              ['Promo2SinceWeek', 'Promo2SinceYear', 'Events', 'StateHoliday',
                       'Precipitationmm', 'is_match', 'is_champ', 'mean_month_sales']
#   + ['Promo2']
   )
for i in l:
    if i in features:
        features.remove(i)
# features.append('from_closed')
# features.append('from_promo')
# features.append('from_school_holiday')
# features.append('from_holiday')
# features.append('till_holiday')

features.append('State')
# features.append('Events')
features.append('Mean_TemperatureC')
features.append('Mean_Sea_Level_PressurehPa')
features.append('Precipitationmm')
# features.append('is_match')
# features.append('is_champ')

In [11]:
# Delete closed stores from validation, this is how I validate
X_test = X_test[X_test.Open == 1]
X_test.reset_index(drop = True, inplace = True)
ind = list(X_test[X_test.Store.isin(test_stores)].index)

In [15]:
X_test.iloc[ind].shape

(35262, 50)

In [13]:
X_train.shape

(747919, 50)

In [14]:
len(features)

27

## Attempt in outlier deletion. Didn't succeed, therefore never launch this cell
Простой способ определить выброс - пробежаться скользящим средним по значениям в Sales. Я ожидаю, что количество выбросов не превышает процента, подгоню параметр скользящего среднего под это значение. 
Формула примерно такая: $x_{ i, t}=\min\left(Rolling\,window_{ i, t},\, x_{ i, t}\right)$

In [66]:
# %%time
coeff = 3.5
num_outliers = 0
num_shops = X_train.Store.nunique()
rolling_window = 10
num_observations = 0
for i in X_train.Store.unique():
    temp_series = pd.rolling_mean(X_train[X_train.Store == i].Sales, rolling_window)
    ind = (X_train[X_train.Store == i].Sales > temp_series*coeff) | (X_train[X_train.Store == i].Sales*coeff < temp_series)
    num_outliers += ind.sum()
    temp_series_2 = pd.rolling_mean(X_train[X_train.Store == i].Sales, 2)
    X_train.loc[(X_train.Store == i) & ind, 'Sales'] = temp_series_2.loc[ind]
#     X_train.iloc[list(ind.index)].loc[ind, 'Sales'] = temp_series_2.loc[ind]
    num_observations += X_train[X_train.Store == i].shape[0]
print('Всего найдено выбросов в {0} магазинах: {1}.\nВсего наблюдений {2}, относительное число выбросов {3}'
      .format(num_shops, num_outliers, num_observations, num_outliers/num_observations))

Всего найдено выбросов в 1115 магазинах: 421.
Всего наблюдений 747919, относительное число выбросов 0.0005628951798256228


In [11]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train[features], X_train["Sales"] )
if validation:
    dvalid = xgb.DMatrix(X_test[features], X_test["Sales"])
else:
    dtest = xgb.DMatrix(X_test[features])

In [67]:
X_test.shape

(45884, 50)

In [23]:
import xgboost as xgb
import datetime
r = datetime.datetime.now()
dtrain = xgb.DMatrix(X_train[features], np.log(X_train["Sales"] + 1))
# dtrain = xgb.DMatrix(X_train[features], X_train["Sales"])
if validation:
    dvalid = xgb.DMatrix(X_test[features], np.log(X_test["Sales"] + 1))
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
else:
    dtest = xgb.DMatrix(X_test[features])

e = True
d = 5
for i in range(d):
    params = {"objective": "reg:linear",
          "learning_rate": 0.061,
          "max_depth": 11,
          "subsample": 0.8,
          "colsample_bytree":0.7,
          "silent": 1,
          'min_child_weight': 11,
        'seed':i + 22,
        'nthread':4}

    num_trees = 700
    if not validation:
        gbm = xgb.train(params, 
                        dtrain, 
                        num_trees, 
#                         evals=watchlist, 
#                         early_stopping_rounds=50, 
#                         feval=rmspe_xg, 
    #                     verbose_eval=True
                       )
    else:
        gbm = xgb.train(params, 
                    dtrain, 
                    num_trees, 
#                         evals=watchlist, 
#                         early_stopping_rounds=50, 
#                         feval=rmspe_xg, 
#                         verbose_eval=False
                   )

    train_probs = gbm.predict(dtrain)
    indices = train_probs < 0
    train_probs[indices] = 0
    error_train = rmspe(np.exp(train_probs) - 1, X_train['Sales'].values)
#     error_train = rmspe(train_probs, X_train['Sales'].values)
    if validation:
        test_probs = gbm.predict(dvalid)
    else:
        test_probs = gbm.predict(dtest)
    indices = test_probs < 0.
    test_probs[indices] = 0
    if validation:
        error = rmspe(np.exp(test_probs[ind]) - 1, X_test.iloc[ind]['Sales'].values)
        print(i, 'Test: ', error,', Train: ', error_train , 'time - ', datetime.datetime.now()- r)
    else:
        print(i, ', Train: ', error_train, ', time - ', datetime.datetime.now()- r)
    if type(e) == bool:
        e_geom = np.log(test_probs.copy())/d
        e = test_probs.copy()/d
    else:
        e += test_probs/d
        e_geom += np.log(test_probs)/d
if validation:
    error_geom = rmspe(np.exp(np.exp(e_geom[ind]))-1, X_test.iloc[ind]['Sales'].values)
    error = rmspe(np.exp(e[ind])-1, X_test.iloc[ind]['Sales'].values)
    print('average ', error, ', time - ', datetime.datetime.now()- r)
    print('geometric average ', error_geom, ', time - ', datetime.datetime.now()- r)
else:
    submission = pd.DataFrame({"Id": X_test["Id"], "Sales": np.exp(e) - 1})
    submission_geom = pd.DataFrame({"Id": X_test["Id"], "Sales": np.exp(np.exp(e_geom)) - 1})
#     submission = pd.DataFrame({"Id": X_test["Id"], "Sales": e})
    submission.to_csv("700_moments_no_football.csv", index=False)
    submission_geom.to_csv("700_moments_no_football_geom.csv", index=False)

0 Test:  0.107043538022 , Train:  0.0815677329763 time -  0:09:13.488695
1 Test:  0.106052421883 , Train:  0.0792847638172 time -  0:18:19.902588
2 Test:  0.107081787974 , Train:  0.0816014501675 time -  0:27:15.563731
3 Test:  0.107587830181 , Train:  0.0792083075691 time -  0:36:19.704748
4 Test:  0.108992415644 , Train:  0.0819524176216 time -  0:45:22.640674
average  0.105843098476 , time -  0:45:22.688845
geometric average  0.105840778341 , time -  0:45:22.688928


In [45]:
from sklearn.ensemble import RandomForestRegressor

In [46]:
rf = RandomForestRegressor(n_estimators=500, 
                           criterion='mse', 
                           max_depth=5, 
                           min_samples_split=2, 
                           min_samples_leaf=1, 
                           min_weight_fraction_leaf=0.0, 
                           max_features=0.6, 
                           max_leaf_nodes=None, 
                           bootstrap=True, 
                           oob_score=False, 
                           n_jobs=-1, 
                           random_state=None, 
                           verbose=0, 
                           warm_start=False)
rf.fit(X_train[features], np.log(X_train['Sales']+1))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features=0.6, max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [47]:
import matplotlib.pyplot as plt 
%matplotlib inline

importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(len(features)):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]), features[indices[f]])

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(len(features)), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(len(features)), indices)
plt.xlim([-1, len(features)])
plt.show()

## Hyperopt.
I average several predictions for the validation set, as this greatly improves the prediction quality of the overall model.

In [ ]:
# %%time
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials 
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
# from sklearn.preprocessing import StandardScaler
import xgboost as xgb
# from sklearn.svm import LinearSVC

import warnings
warnings.filterwarnings("ignore")

def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe


def objective(space):

#     clf = RandomForestClassifier(n_estimators=50,
#                                 max_features = space['max_features'], 
#                                  max_depth = space['max_depth'],
#                                  min_samples_leaf = space['min_samples_leaf'],
#                                 n_jobs = -1)
#################
    loss = 0.
    pred = True
    for i in range(4):
        params = {"objective": "reg:linear",
                  "learning_rate": space['learning_rate'],
                  "max_depth": space['max_depth'],
                  "subsample": space['subsample'],
                  "colsample_bytree": space['colsample_bytree'],
                  "silent": 1,
                  'min_child_weight': space['min_child_weight'],
            'seed':i*100+22,
            'nthread':4
                  }
        
#         clf = xgb.XGBRegressor(n_estimators = 1000, missing = -1., 
#                                 max_depth = space['max_depth'],
#                                 min_child_weight = space['min_child_weight'],
#                                 subsample = space['subsample'],
#                                 colsample_bytree = space['colsample_bytree'],
#                                 learning_rate = space['learning_rate'],
#                                nthread = 2,
#                                objective = 'reg:linear')
        num_trees = 1000
        gbm = xgb.train(params, 
                        dtrain, 
                        num_trees, 
#                         evals=watchlist, 
#                         early_stopping_rounds=50, 
                        feval=rmspe_xg, 
#                         verbose_eval=True
                       )
        test_probs = gbm.predict(dvalid)
        indices = test_probs < 0
        test_probs[indices] = 0
        min_error = rmspe(test_probs, X_test['Sales'].values)
        train_probs = gbm.predict(dtrain)
        indices = train_probs < 0
        train_probs[indices] = 0
        error_train = rmspe(train_probs, X_train['Sales'].values)
        if type(pred) == bool:
            pred = test_probs.copy()/4.
        else:
            pred += test_probs/4.
        print(min_error, "train_error:", error_train,
              space['learning_rate'], space['max_depth'], 
              space['min_child_weight'], space['subsample'],
              space['colsample_bytree'], i)
        if min_error>108:
            print('Big error: {0}'.format(min_error))
            min_error = 0.3
            break
    if min_error != 0.3:
        min_error = rmspe(pred, X_test['Sales'].values)
#         error = rmspe(np.exp(pred/4.) - 1, X_test['Sales'].values)
        print('Obtained error {0}'.format(min_error))
        with open('no_logs.txt', 'a') as f:
            f.write(str(min_error) + ' ' + str(space['learning_rate']) + ' ' + str(space['max_depth']) + ' ' +
                   str(space['min_child_weight']) + ' ' + str(space['subsample']) + ' ' + str(space['colsample_bytree']))
    else:
        print('error too big')
    return{'loss':min_error, 'status': STATUS_OK } 

###### RF parameter space #######
# space ={
#         'max_depth': hp.quniform("x_max_depth", 5, 30, 1),
#         'min_samples_leaf': hp.quniform ('min_samples_leaf', 1, 10, 1),
#         'max_features': hp.uniform ('max_features', 0.5, 0.9)
#     }

###### XGB parameter space #######
space ={
        'max_depth': hp.quniform("max_depth", 8, 16, 1),
        'min_child_weight': hp.quniform ('min_child_weight', 3, 15, 2),
        'subsample': hp.uniform ('subsample', 0.4, 0.9),
            'colsample_bytree': hp.uniform ('colsample_bytree', 0.4, 0.9),
    'learning_rate': hp.uniform('lr', 0.02, 0.08)
    }

###### LinearSVC parameter space #######
# space ={
#         'C': hp.uniform ('C', 0.001, 5),
#         'intercept_scaling': hp.uniform ('intercept_scaling', 0.001, 10),
#         'dual': hp.choice('dual', [True, False])
# #         'penalty': hp.choice('penalty', ['l1', 'l2'])
   
#     }
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=75,
            trials=trials)

print(best)